In [2]:
import graphlab
import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# %matplotlib notebook

from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

%pylab inline

This non-commercial license of GraphLab Create for academic use is assigned to srinivas.sai@sjsu.edu and will expire on November 28, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1511860459.log


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [50]:
dataset1 = '/home/user/257/10000.txt'
dataset2 = '/home/user/257/song_data.csv'

temp_df1 = pd.read_csv(dataset1, sep="\t", names =["user_id", "song_id", "listen_count"])
temp_df2 = pd.read_csv(dataset2)
#temp_df1.head()
#temp_df2.head()


final_df = pd.merge(temp_df1, temp_df2.drop_duplicates(['song_id']), on="song_id", how="left")
final_df = graphlab.SFrame(final_df)
final_df.head()

user_id,song_id,listen_count,title,release
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Graduation
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left ToLose ...
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Antología Audiovisual
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,The Fame Monster
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,There Is Nothing Left ToLose ...
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Musik von Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Hôtel Costes 7 byStéphane Pompougnac ...


In [54]:
final_df = final_df.head(10000)

#Merge song title and artist_name columns to make a merged column
final_df['song'] = final_df['title'].apply(str) + " - " + final_df['artist_name'].apply(str)

In [55]:
final_df.head()

user_id,song_id,listen_count,title,release
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Graduation
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left ToLose ...
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Antología Audiovisual
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,The Fame Monster
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,There Is Nothing Left ToLose ...
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Musik von Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Hôtel Costes 7 byStéphane Pompougnac ...


In [68]:
users = final_df['user_id'].unique()

In [69]:
train_data,test_data = final_df.random_split(.8,seed=0)

In [70]:
train_data.head()

user_id,song_id,listen_count,title,release
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Graduation
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left ToLose ...
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Antología Audiovisual
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,The Fame Monster
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,There Is Nothing Left ToLose ...
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Musik von Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Hôtel Costes 7 byStéphane Pompougnac ...


In [71]:
popularity_model = graphlab.popularity_recommender.create(train_data, user_id='user_id', item_id='song')

Recsys training: model = popularity

Warning: Ignoring columns song_id, listen_count, title, release, artist_name, year;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

Data has 8032 observations with 365 users and 4511 items.

Data prepared in: 0.037842s

8032 observations to process; with 4511 unique items.

In [72]:
popularity_model.recommend(users=[users[0]])

user_id,song,score,rank
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Sehr kosmisch - Harmonia,35.0,1
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Undo - Björk,27.0,2
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Secrets - OneRepublic,25.0,3
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Dog Days Are Over (RadioEdit) - Florence + The ...,25.0,4
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,The Scientist - Coldplay,23.0,5
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,You're The One - DwightYoakam ...,23.0,6
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Use Somebody - Kings OfLeon ...,22.0,7
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Lucky (Album Version) -Jason Mraz & Colbie ...,19.0,8
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Fireflies - CharttraxxKaraoke ...,19.0,9
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Revelry - Kings Of Leon,18.0,10


In [73]:
popularity_model.recommend(users=[users[1]])

user_id,song,score,rank
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Sehr kosmisch - Harmonia,35.0,1
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Undo - Björk,27.0,2
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Secrets - OneRepublic,25.0,3
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Dog Days Are Over (RadioEdit) - Florence + The ...,25.0,4
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,The Scientist - Coldplay,23.0,5
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,You're The One - DwightYoakam ...,23.0,6
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Use Somebody - Kings OfLeon ...,22.0,7
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Lucky (Album Version) -Jason Mraz & Colbie ...,19.0,8
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Fireflies - CharttraxxKaraoke ...,19.0,9
5d5e0142e54c3bb7b69f548c2ee55066c90700eb ...,Revelry - Kings Of Leon,18.0,10


In [74]:
eval = popularity_model.evaluate(test_data)


Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    | 0.0369230769231 | 0.0127582417582 |
|   2    | 0.0323076923077 | 0.0195659340659 |
|   3    | 0.0235897435897 | 0.0202838827839 |
|   4    | 0.0261538461538 | 0.0255843114574 |
|   5    | 0.0264615384615 | 0.0305382007054 |
|   6    | 0.0235897435897 | 0.0313676659903 |
|   7    | 0.0237362637363 | 0.0343050717218 |
|   8    | 0.0223076923077 | 0.0354629664586 |
|   9    | 0.0215384615385 | 0.0368548925402 |
|   10   | 0.0209230769231 | 0.0382948385591 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

